## Get Albums and Artist Info from Spotify

In [6]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json
import re
import pandas as pd
import numpy as np
import sys

In [3]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json
import re
import pandas as pd
import numpy as np
import sys

#These are my personal spotify credentials to be able to access the spotify API and ultimately access their data
clientId = #Input own
clientSecret = #Input own
clientCredentialsManager = SpotifyClientCredentials(client_id=clientId, client_secret=clientSecret)
sp = spotipy.Spotify(client_credentials_manager=clientCredentialsManager) #spotify object to access API

def spotifyArtistInfo(artist):
    result = sp.search(artist,type='artist') #search query
    artist = result['artists']['items'][0]
    followers = artist['followers']['total']
    genres = artist['genres']
    return (followers, genres)

def spotifyArtistAblums(artist):
    search = sp.search(q='artist:' + artist, type='artist')
    artist = search['artists']['items'][0]
    albums = []
    search = sp.artist_albums(artist['id'], album_type='album')
    albums.extend(search['items'])
    while search['next']:
        search = sp.next(search)
        albums.extend(search['items'])
    studioAlbums = []
    allAlbums = []
    albumNames = set()
    albums.sort(key=lambda album:album['name'].lower())
    firstAlbYear = 3000 #Arbitrarilly large
    for album in albums:
        name = album['name']
        albumId = album['id']
        year = int(album['release_date'][0:4])
        if year < firstAlbYear:
            firstAlbYear = year
        type_ = album['album_group']
        #If there is an album that has a deluxe version, the deluxe one will be selected
        exclude = ["spotify sessions","spotify exclusive","bbc session", "muve sessions","atlantic sessions","eastwest sessions","radio release", "karaoke","extended edition", "commentary", " tour edition", "remix album", "standard version", "international edition","international version", "international deluxe", "anniversary edition", "special edition", " case version", "bonus version", "asia deluxe", "japan deluxe", "china version", "japan version", "japanese version","norwegian version", "us version","uk version","canada version", "int'l version","(international", "international)", "soundtrack", "radio release", "deluxe package", "(clean)", "clean version", "- clean", "remastered", "remaster", "stereo mix", "mono mix", "best of", "greatest hits", "greatest tracks", "greatest hitz","ultimate collection", "hits collection", "re-release", "remixes", "remix version", "remix album", " remix","instrumental", "(deluxe clean)", "ringtone", "th anniversary", "audiobook", "track by track", "track-by-track"]
        if name.lower() not in albumNames and not any(x in name.lower() for x in exclude) and not ("edition" in name.lower() and "deluxe" not in name.lower()) and type_ == 'album':
            albumNames.add(name.lower())
            allAlbums.append((name, album))
    notDeluxe = []
    deluxeVersion = []
    unDeluxed = []
    terms = ["(deluxe)", "(super deluxe edition)", "(super deluxe version)", "super deluxe edition","super deluxe version" ,"(deluxe edition)", "(deluxe version)", " deluxe edition", " deluxe version", "deluxe","(bonus tracks version)", "(bonus track version)", "(bonus version)", "bonus tracks", "bonus track", "bonus version", "(extended versions)", "(extended version)", "(extended edition)", "extended version", "extended edition", "extended", "(explicit version)", "(explicit)", "explicit version", "explicit", " (edited version)", "(edited)", " edited version"]
    for a in allAlbums:
        if "deluxe" in a[0].lower() or "bonus" in a[0].lower() or "extended" in a[0].lower() or "explicit" in a[0].lower() or "edited" in a[0].lower():
            deluxeVersion.append(a)
            studioAlbums.append(a[1])
            deluxeName = a[0].lower()
            for term in terms:
                deluxeName = deluxeName.replace(term, "")
            unDeluxed.append(deluxeName)
        else:
            notDeluxe.append(a)
    for item in notDeluxe:
        if not any(d.strip().lower() == item[0].strip().lower() for d in unDeluxed):
            studioAlbums.append(item[1])
    for i in studioAlbums:
        #Toggle this on to print
        #print(i["name"])
        pass
    return (studioAlbums, firstAlbYear)
        

SyntaxError: invalid syntax (<ipython-input-3-9ba5af16211d>, line 10)

In [3]:
albs, firstYear = spotifyArtistAblums("ne")
print(len(albs))

9


## Run Qualifier Check for All Billboard Artists

An artist qualifies for consideration in the study if:
- they have content which has appeared on the Billboard Hot 100 in the last 20 years
- they they have at least 3 albums released
- their first album was released at least 7 years ago

The billboard scraper goes back 1043 weeks (20 years) to collect data, and then this script will use spotify to ultimately see if the artist qualifies.

In [3]:
#Currently doesn't catch destiny's child
dataPre = pd.DataFrame(pd.read_csv('../Data/Weekly_data_clean.csv'))
artistData = pd.DataFrame(columns = ["Artist", 'Followers', "Genres", "NumAlbums", "YearFirstAlbum"])
drops = []
outArtists = []
for index, row in dataPre.iterrows():
    artists = row["Artists"].split(",")
    ##Some special cases
    if artists == ['Machine Gun Kelly', '', ' Ambassadors', ' Bebe Rexha']:
        artists = ['Machine Gun Kelly', 'X Ambassadors', ' Bebe Rexha']
    elif artists == ["Mumford","Sons"]:
        artists=["Mumford and Sons"]
    elif artists == ['Pusha T Kanye West']:
        artists = ["Pusha T", "Kanye West"]
    elif artists == ['Jay Z Kanye West']:
        artists = ['Jay Z', "Kanye West"]
    elif 'Tyler' in artists and ' The Creator' in artists:
        artists.remove("Tyler")
        artists.remove(" The Creator")
        artists.append("Tyler The Creator")
    elif " Swae Lee Or Rae Sremmurd" in artists:
        artists = ["Swae Lee", "Rae Sremmurd"]
    elif "Plies Or OJ Da Juiceman" in artists:
        artists = ['Plies', 'OJ Da Juiceman']
    elif "The Rock Heroes" in artists:
        continue
    if all(a.strip() in (artistData['Artist'].tolist() + outArtists) for a in artists):
        for a in artists:
            if a.strip() in outArtists:
                drops.append(index)
        continue
    else:
        for a in artists:
            artist = a.strip().replace(" &", "")
            if artist == "Pro":
                continue
            elif " + " in artist and (artist !=  "Dan + Shay" and artist != "Florence + The Machine"):
                new = artist.split("+")
                artists.extend(new)
                continue
            elif " with " in artist:
                new = artist.split(' with ')
                artists.extend(new)
                continue
            elif " With " in artist:
                new = artist.split(' With ')
                artists.extend(new)
                continue
            if artist in outArtists:
                drops.append(index)
                continue
            if artist.lower() not in map(lambda x:x.lower(),artistData['Artist'].tolist()):
                print(str(index) + " ARTIST : " + artist)
                try:
                    albums, firstYear= spotifyArtistAblums(artist)
                except: 
                    #Add to this as we find
                    if artist == "Soulja Boy Tell'em" and "Soulja Boy" not in artistData['Artist'].tolist():
                        artist = "Soulja Boy"
                    elif artist == "Ludacris Co" and "Ludacris" not in artistData['Artist'].tolist():
                        artist = "Ludacris"
                    else:
                        continue
                    try:
                        albums, firstYear= spotifyArtistAblums(artist)
                    except:
                        print("Not actually a spotify artist!")
                        continue
                if len(albums) >= 3 and firstYear < 2012: #then they qualify, and collect the artist data
                    followers, genres = spotifyArtistInfo(artist)
                    artistData = artistData.append({"Artist": artist, 'Followers':followers, "Genres": ",".join(genres), "NumAlbums":len(albums), "YearFirstAlbum": firstYear}, ignore_index = True) 
                else:
                    outArtists.append(artist)
                    drops.append(index)
        
dataPost = dataPre.drop(drops)
dataPost.to_csv("../Data/billboardRankings.csv")
artistData.to_csv("../Data/artistInfo.csv")
        
        

0 ARTIST : Lil Nas
0 ARTIST : 
1 ARTIST : Shawn Mendes
1 ARTIST : Camila Cabello
2 ARTIST : Billie Eilish
3 ARTIST : Khalid
4 ARTIST : Ed Sheeran
4 ARTIST : Justin Bieber
5 ARTIST : Jonas Brothers
6 ARTIST : DaBaby
7 ARTIST : Drake
8 ARTIST : Chris Brown
9 ARTIST : Post Malone
10 ARTIST : Lizzo
11 ARTIST : Swae Lee
12 ARTIST : Taylor Swift
14 ARTIST : Sam Smith
14 ARTIST : Normani
15 ARTIST : Lil Nas X
16 ARTIST : Halsey
17 ARTIST : Blake Shelton
18 ARTIST : Polo G
19 ARTIST : Nicki Minaj
20 ARTIST : Ava Max
21 ARTIST : Cardi B
22 ARTIST : Ariana Grande
23 ARTIST : Panic! At The Disco
24 ARTIST : Marshmello
24 ARTIST : Bastille
25 ARTIST : Lee Brice
26 ARTIST : Katy Perry
27 ARTIST : Morgan Wallen
28 ARTIST : Blanco Brown
29 ARTIST : Young Thug
29 ARTIST : J. Cole
29 ARTIST : Travis Scott
32 ARTIST : Luke Combs
33 ARTIST : City Girls
34 ARTIST : benny blanco
36 ARTIST : Daddy Yankee
37 ARTIST : Mustard
37 ARTIST : Migos
39 ARTIST : Meek Mill
41 ARTIST : Lady Gaga
41 ARTIST : Bradley Co

# Attaining Spotify's Audio Features

In [4]:
def trackInfo(albums, artist):
    albumData = pd.DataFrame(columns = ['Artist', 'Album','Name', 'Popularity', 'Duration', 'Explicit', 'Danceability', 'Energy', 'Loudness', 'Mode', 'Speechiness', 'Acousticness', 'Instrumentalness', 'Liveness', 'Valence', 'Tempo', 'TimeSignature'])
    for album in albums:
        albumId = album['id']
        albumName = album['name']
        tracks = sp.album_tracks(albumId)['items']
        for track in tracks:
            trackName = track['name']
            features = sp.audio_features(track['uri'])
            #to get popularity and explicit
            trackInfo = sp.track(track['uri'])
            #Create DF row     ###Maybe try to change duration to be in seconds
            albumData = albumData.append({'Artist':artist, 'Album':albumName,'Name':trackName, 'Popularity':trackInfo['popularity'], 'Duration':features[0]['duration_ms'], 'Explicit':trackInfo['explicit'], 'Danceability':features[0]['danceability'], 'Energy':features[0]['energy'], 'Loudness':features[0]['loudness'], 'Mode':features[0]['mode'], 'Speechiness':features[0]['speechiness'], 'Acousticness':features[0]['acousticness'], 'Instrumentalness':features[0]['instrumentalness'], 'Liveness':features[0]['liveness'], 'Valence':features[0]['valence'], 'Tempo':features[0]['tempo'], 'TimeSignature':features[0]['time_signature']}, ignore_index = True)
    return albumData

In [5]:
trackInfo(albs, 'Ed Sheeran')

2018-06-08
2015-02-16
2012-01-01
2019-10-11
2007-01-01
2006-01-01
2010-01-01
2010-11-02
2008-01-01


,Artist,Album,Name,Popularity,Duration,Explicit,Danceability,Energy,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,TimeSignature
0,Ed Sheeran,GOOD MAN (Deluxe),“Caterpillars 1st” (INTRO),32,41000,True,0.684,0.523,-13.908,1,0.7230,0.63500,0,0.4800,0.869,86.957,4
1,Ed Sheeran,GOOD MAN (Deluxe),1 MORE SHOT,42,230507,False,0.710,0.785,-5.460,1,0.1440,0.48600,1.18e-06,0.5000,0.737,119.979,4
2,Ed Sheeran,GOOD MAN (Deluxe),LA NIGHTS,53,234840,False,0.738,0.546,-6.637,0,0.0416,0.13900,0,0.1920,0.198,100.014,4
3,Ed Sheeran,GOOD MAN (Deluxe),NIGHTS LIKE THESE,49,204453,False,0.494,0.729,-4.934,1,0.1610,0.13300,0,0.1430,0.421,78.833,4
4,Ed Sheeran,GOOD MAN (Deluxe),U DESERVE,40,224973,False,0.620,0.513,-8.811,0,0.0599,0.06590,0.000364,0.1070,0.359,97.373,4
5,Ed Sheeran,GOOD MAN (Deluxe),SUMMERTIME,35,174933,False,0.632,0.722,-6.917,0,0.0674,0.02780,0.117,0.0828,0.551,170.043,4
6,Ed Sheeran,GOOD MAN (Deluxe),PUSH BACK,58,221280,False,0.774,0.688,-5.088,0,0.0459,0.17800,4e-06,0.0701,0.745,107.011,4
7,Ed Sheeran,GOOD MAN (Deluxe),BREATHE,48,243787,False,0.369,0.462,-8.302,0,0.1430,0.19500,6.63e-05,0.1290,0.336,180.218,3
8,Ed Sheeran,GOOD MAN (Deluxe),ON UR MIND,43,202253,True,0.481,0.591,-4.738,1,0.0682,0.00985,3.29e-06,0.1360,0.410,116.903,4
9,Ed Sheeran,GOOD MAN (Deluxe),BACK CHAPTERS,37,250533,False,0.550,0.618,-5.162,0,0.0632,0.01930,0.00274,0.1220,0.279,125.098,4


## Write the Files For Archetype Artists

In [4]:
def archetypeContentInfo(artistList):
    df = pd.DataFrame(columns = ['Artist', 'Album', 'Popularity', 'Explicit', 'Danceability', 'Energy', 'Loudness', 'Mode', 'Speechiness', 'Acousticness', 'Instrumentalness', 'Liveness', 'Valence', 'Tempo', 'Duration', 'TimeSignature'])
    for artist in artistList:
        albums, firstYear = spotifyArtistAblums(artist)
        print("on artist " + str(artistList.index(artist)) + "out of 1168")
        try:
            data = trackInfo(albums, artist)
            df = pd.concat([data, df])
        except:
            pass
    df.to_csv("../Data/songAttributes.csv")
    

In [4]:
#built this separate to not recollect all this data
def archetypeAlbsDateGenre(artistList):
    data = pd.DataFrame(columns = ['Artist', 'Album', 'ReleaseDate', 'Genres'])
    for artist in artistList:
        try:
            albums, firstYear = spotifyArtistAblums(artist)
            print("on artist " + str(artistList.index(artist)) + "out of 1168")
            try: 
                for album in albums:
                    print('hi')
                    print(album['genres'])
                    new = {"Artist": artist, "Album": album["name"], "ReleaseDate":album["release_date"], "Genres":album["genres"]}
                    data = data.append(new, ignore_index= True)
            except:
                pass
        except:
            pass
    data.to_csv("../Data/albReleaseDatesGenres.csv")

In [7]:
artistList = pd.DataFrame(pd.read_csv("../Data/artistInfo.csv",names = ["Artist", "Followers", "Genres", "NumAlbums", "YearFirstAlbum", "Gender", "Group/Solo"])).Artist.to_list()[1:]

In [8]:
archetypeAlbsDateGenre(artistList)

In [5]:

archetypeContentInfo(artistList)

on artist 0out of 1168


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


on artist 1out of 1168
on artist 2out of 1168
on artist 3out of 1168
on artist 4out of 1168
on artist 5out of 1168
on artist 6out of 1168
on artist 7out of 1168
on artist 8out of 1168
on artist 9out of 1168
on artist 10out of 1168
on artist 11out of 1168
on artist 12out of 1168
on artist 13out of 1168
on artist 14out of 1168
on artist 15out of 1168
on artist 16out of 1168
on artist 17out of 1168
on artist 18out of 1168
on artist 19out of 1168
on artist 20out of 1168
on artist 21out of 1168
on artist 22out of 1168
on artist 23out of 1168
on artist 24out of 1168
on artist 25out of 1168
on artist 26out of 1168
on artist 27out of 1168
on artist 28out of 1168
on artist 29out of 1168
on artist 30out of 1168
on artist 31out of 1168
on artist 32out of 1168
on artist 33out of 1168
on artist 34out of 1168
on artist 35out of 1168
on artist 36out of 1168
on artist 37out of 1168
on artist 38out of 1168
on artist 39out of 1168
on artist 40out of 1168
on artist 41out of 1168
on artist 42out of 1168
o